In [ ]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
from flask import Flask, request
from flask_restful import Api, Resource, reqparse

app = Flask(__name__)
api = Api(app)

class Admin(Resource):

    def post(self):
        
        data = request.get_json() #get data from request
        url = data['url']
        
        reqs = requests.get(url)
        soup = BeautifulSoup(reqs.text, 'lxml') #load the requested url to bs4

        headings=[]
        contents=[]
        temp = []
        count=0
        
        headings.append("Drug name is:") 
        contents.append(soup.find("h1").text) #extracting the drug name

        for heading in soup.find_all( ["h2","p"]): #loop will go through every h2 and p tags
            if(heading.name == "h2"): #this condition will executes if it finds a h2 tag
                count+=1
                if (count == 11):
                    break
                headings.append(heading.text)
                if(temp != []):
                    contents.append(temp)
                temp = []

            if(heading.name == "p"): 
                temp.append(heading.text)

        contents.pop(1)
        headings.pop(10)

        df = pd.DataFrame({'subheadings':headings,'content':contents})
        result = df.to_json(contents[0]+".json",orient="records")
        result = df.to_json(orient="records")
        parsed = json.loads(result)
        json.dumps(parsed)
        
        return parsed, 200

# Add URL endpoints
api.add_resource(Admin, '/url')

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Aug/2021 16:52:04] "POST /url HTTP/1.1" 200 -
127.0.0.1 - - [04/Aug/2021 16:52:59] "POST /url HTTP/1.1" 200 -
